In [0]:
import pandas as pd
from pyspark.sql.types import ArrayType, StructType
from pyspark.sql.functions import explode_outer, col, arrays_zip
import os
from pyspark.sql.functions import pandas_udf, explode

In [0]:
# File location and type
file_location = "/FileStore/tables/Patient_1.ndjson"
file_type = "ndjson"

In [0]:
dbutils.fs.ls("dbfs:" + file_location)
df = spark.read.json("dbfs:" + file_location)
display(df)

active,address,birthDate,communication,deceasedDateTime,extension,gender,id,identifier,maritalStatus,meta,multipleBirthBoolean,name,resourceType,telecom,text
true,null,1960-12-25,null,null,null,male,cb26aa5c-b5e9-4b8b-963e-902ffecf0154,null,null,"List(2022-01-31T21:43:30.008+00:00, 1)",null,"List(List(Kirk, List(James, Tiberious), null, official), List(null, List(Jim), null, usual))",Patient,null,null
null,"List(List(Sharon, US, List(List(List(List(latitude, 42.1408839404588), List(longitude, -71.22137721510623)), http://hl7.org/fhir/StructureDefinition/geolocation)), List(734 Bernhard Manor), 02067, Massachusetts))",1919-11-10,"List(List(List(List(List(en-US, English, urn:ietf:bcp:47)), English)))",null,"List(List(http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName, null, null, Micah422 Reichert620), List(http://hl7.org/fhir/StructureDefinition/patient-birthPlace, List(Boston, US, Massachusetts), null, null), List(http://synthetichealth.github.io/synthea/disability-adjusted-life-years, null, 0.0, null), List(http://synthetichealth.github.io/synthea/quality-adjusted-life-years, null, 102.0, null))",female,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,"List(List(https://github.com/synthetichealth/synthea, null, 52271ada-ad3d-02e5-6bf8-7d01b38a9c84), List(http://hospital.smarthealthit.org, List(List(List(MR, Medical Record Number, http://terminology.hl7.org/CodeSystem/v2-0203)), Medical Record Number), 52271ada-ad3d-02e5-6bf8-7d01b38a9c84), List(http://hl7.org/fhir/sid/us-ssn, List(List(List(SS, Social Security Number, http://terminology.hl7.org/CodeSystem/v2-0203)), Social Security Number), 999-66-4944), List(urn:oid:2.16.840.1.113883.4.3.25, List(List(List(DL, Driver's License, http://terminology.hl7.org/CodeSystem/v2-0203)), Driver's License), S99970616), List(http://standardhealthrecord.org/fhir/StructureDefinition/passportNumber, List(List(List(PPN, Passport Number, http://terminology.hl7.org/CodeSystem/v2-0203)), Passport Number), X50552415X))","List(List(List(M, M, http://terminology.hl7.org/CodeSystem/v3-MaritalStatus)), M)","List(2022-02-02T22:54:01.944+00:00, 2)",false,"List(List(Pollich983, List(Deneen201), List(Mrs.), official), List(Conn188, List(Deneen201), List(Mrs.), maiden))",Patient,"List(List(phone, home, 555-905-8926))","List(Generated by Synthea.Version identifier: master-branch-latest . Person seed: -6061321512527560083 Population seed: 1643834518691, generated)"
null,"List(List(Fall River, US, List(List(List(List(latitude, 41.67988181138803), List(longitude, -71.15989157955373)), http://hl7.org/fhir/StructureDefinition/geolocation)), List(749 Lindgren Center), 02747, Massachusetts))",2000-01-22,"List(List(List(List(List(en-US, English, urn:ietf:bcp:47)), English)))",null,"List(List(http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName, null, null, Daniella68 Breitenberg711), List(http://hl7.org/fhir/StructureDefinition/patient-birthPlace, List(Marion, US, Massachusetts), null, null), List(http://synthetichealth.github.io/synthea/disability-adjusted-life-years, null, 0.0, null), List(http://synthetichealth.github.io/synthea/quality-adjusted-life-years, null, 21.0, null))",male,94ca1022-8ef5-798c-eb1e-1706ad4f5807,"List(List(https://github.com/synthetichealth/synthea, null, 94ca1022-8ef5-798c-eb1e-1706ad4f5807), List(http://hospital.smarthealthit.org, List(List(List(MR, Medical Record Number, http://terminology.hl7.org/CodeSystem/v2-0203)), Medical Record Number), 94ca1022-8ef5-798c-eb1e-1706ad4f5807), List(http://hl7.org/fhir/sid/us-ssn, List(List(List(SS, Social Security Number, http://terminology.hl7.org/CodeSystem/v2-0203)), Social Security Number), 999-55-2895), List(urn:oid:2.16.840.1.113883.4.3.25, List(List(List(DL, Driver's License, http://terminology.hl7.org/CodeSystem/v2-0203)), Driver's License), S99969631), List(http://standardhealthrecord.org/fhir/StructureDefinition/passportNumber, List(List(List(PPN, Passport Number, http://terminology.hl7.org/CodeSystem/v2-0203)), Passport N

In [0]:
'''
Transforms a data frame which contains struct columns into a data frame where
each such column is expanded into multiple columns, one for each unique value
in the original struct. Columns are prefaced with the parent name for organization.
'''
def flatten_structs(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:
        # Grab columns that are already flat and move to final df
        parents, df = stack.pop()
        flat_cols = [col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],))) for c in df.dtypes if c[1][:6] != "struct"]
        columns.extend(flat_cols)
        
        # Recursively collect columns within each struct, searching until
        # the projection is flat
        nested_cols = [c[0] for c in df.dtypes if c[1][:6] == "struct" ]    
        for nc in nested_cols:
            projected_df = df.select(nc + ".*")
            stack.append((parents + (nc,), projected_df))
            
    return nested_df.select(columns)

In [0]:
'''
Transforms a spark dataframe into an "exploded" data frame. To explode a data frame 
means to add a new row containing identical identifying information to a base row 
for each array-element encountered in columns whose fields are structs. The result
is returned as a dataframe.
'''
def explode_arrays(df):
    flat_cols = [field.name for field in df.schema.fields if type(field.dataType) != ArrayType]
    struct_cols = [field.name for field in df.schema.fields if type(field.dataType) == ArrayType]
    
    # Add new rows for each array element, then select and join with columns
    # that are already flat
    exploded_df = df.withColumn('vals', explode_outer(arrays_zip(*struct_cols))) \
           .select(*flat_cols,'vals.*') \
           .fillna('', subset=struct_cols)
    return exploded_df

In [0]:
'''
A wrapper function to perform full flattening of an input spark data frame.
Struct columns are separated out into multiple columns, and then the intermediate
expanded data frame is exploded so that each new element encountered generates a 
new row.
'''
def flatten_df(dfflat):
    while len([field.name for field in dfflat.schema.fields if type(field.dataType) == StructType or type(field.dataType) == ArrayType ]) !=0 :
        dfflat = flatten_structs(dfflat)
        dfflat = explode_arrays(dfflat)
    return dfflat

In [0]:
'''
Generates a DDL "create-table" string from a flattened spark dataframe. The result
of this function is a statement of the form:

  CREATE TABLE [table_name] (
    [field1] nvarchar(50),
    [field2] nvarchar(50),
    ...
  )
  
For type safety reasons, all fields in the created DDL are saved as nvarchar(50).
'''
def generate_ddl(df, tblname):
    createtbl = 'CREATE TABLE [' + tblname +  '] ( \n'
    num_columns = len(dfflat.columns)
    for i, y in enumerate(dfflat.columns):
        if i == num_columns-1:
            column_name = '\t['+ y + '] nvarchar(50)); \n\n'
        else:
            column_name = '\t[' + y + '] nvarchar(50), \n'
        createtbl += column_name
    return createtbl

In [0]:
dfflat = flatten_df(df)
display(dfflat)

active,birthDate,deceasedDateTime,gender,id,multipleBirthBoolean,resourceType,text_div,text_status,meta_lastUpdated,meta_versionId,maritalStatus_text,maritalStatus_coding_code,maritalStatus_coding_display,maritalStatus_coding_system,telecom_system,telecom_use,telecom_value,name_family,name_use,identifier_system,identifier_value,identifier_type_text,extension_url,extension_valueDecimal,extension_valueString,extension_valueAddress_city,extension_valueAddress_country,extension_valueAddress_state,communication_language_text,address_city,address_country,address_postalCode,address_state,name_given,name_prefix,address_line,address_extension_url,communication_language_coding_code,communication_language_coding_display,communication_language_coding_system,identifier_type_coding_code,identifier_type_coding_display,identifier_type_coding_system,address_extension_extension_url,address_extension_extension_valueDecimal
true,1960-12-25,null,male,cb26aa5c-b5e9-4b8b-963e-902ffecf0154,null,Patient,null,null,2022-01-31T21:43:30.008+00:00,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,,,,null,null,null,null,null,null,null,null,null
null,1919-11-10,null,female,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,false,Patient,Generated by Synthea.Version identifier: master-branch-latest . Person seed: -6061321512527560083 Population seed: 1643834518691,generated,2022-02-02T22:54:01.944+00:00,2,M,M,M,http://terminology.hl7.org/CodeSystem/v3-MaritalStatus,phone,home,555-905-8926,Pollich983,official,https://github.com/synthetichealth/synthea,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,null,http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName,null,Micah422 Reichert620,null,null,null,English,Sharon,US,02067,Massachusetts,,,,null,null,null,null,null,null,null,null,null
null,1919-11-10,null,female,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,false,Patient,Generated by Synthea.Version identifier: master-branch-latest . Person seed: -6061321512527560083 Population seed: 1643834518691,generated,2022-02-02T22:54:01.944+00:00,2,M,null,null,null,null,null,null,Conn188,maiden,http://hospital.smarthealthit.org,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,Medical Record Number,http://hl7.org/fhir/StructureDefinition/patient-birthPlace,null,null,Boston,US,Massachusetts,null,null,null,null,null,,,,null,null,null,null,null,null,null,null,null
null,1919-11-10,null,female,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,false,Patient,Generated by Synthea.Version identifier: master-branch-latest . Person seed: -6061321512527560083 Population seed: 1643834518691,generated,2022-02-02T22:54:01.944+00:00,2,M,null,null,null,null,null,null,null,null,http://hl7.org/fhir/sid/us-ssn,999-66-4944,Social Security Number,http://synthetichealth.github.io/synthea/disability-adjusted-life-years,0.0,null,null,null,null,null,null,null,null,null,,,,null,null,null,null,null,null,null,null,null
null,1919-11-10,null,female,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,false,Patient,Generated by Synthea.Version identifier: master-branch-latest . Person seed: -6061321512527560083 Population seed: 1643834518691,generated,2022-02-02T22:54:01.944+00:00,2,M,null,null,null,null,null,null,null,null,urn:oid:2.16.840.1.113883.4.3.25,S99970616,Driver's License,http://synthetichealth.github.io/synthea/quality-adjusted-life-years,102.0,null,null,null,null,null,null,null,null,null,,,,null,null,null,null,null,null,null,null,null
null,1919-11-10,null,female,52271ada-ad3d-02e5-6bf8-7d01b38a9c84,false,Patient,Generated by Synthea.Version identifier: master-branch-latest . Person seed: -6061321512527560083 Population seed: 1643834518691,generated,2022-02-02T22:54:01.944+00:00,2,M,null,null,null,null,null,null,null,null,http://standardhealthrecord.org/fhir/StructureDefinition/passportNumber,X50552415X,Passport Number,null,null,null,null,null,null,null,null,null,null,null,,,,null,null,null,null,null,null,null,null,null
null,2000-01-22,null,male,94ca1022-8ef5-798c-eb1e-1706ad4f5807,false,Pa

In [0]:
ddl = generate_ddl(dfflat, "patient")
print(ddl)

CREATE TABLE [patient] ( 
	[active] nvarchar(50), 
	[birthDate] nvarchar(50), 
	[deceasedDateTime] nvarchar(50), 
	[gender] nvarchar(50), 
	[id] nvarchar(50), 
	[multipleBirthBoolean] nvarchar(50), 
	[resourceType] nvarchar(50), 
	[text_div] nvarchar(50), 
	[text_status] nvarchar(50), 
	[meta_lastUpdated] nvarchar(50), 
	[meta_versionId] nvarchar(50), 
	[maritalStatus_text] nvarchar(50), 
	[maritalStatus_coding_code] nvarchar(50), 
	[maritalStatus_coding_display] nvarchar(50), 
	[maritalStatus_coding_system] nvarchar(50), 
	[telecom_system] nvarchar(50), 
	[telecom_use] nvarchar(50), 
	[telecom_value] nvarchar(50), 
	[name_family] nvarchar(50), 
	[name_use] nvarchar(50), 
	[identifier_system] nvarchar(50), 
	[identifier_value] nvarchar(50), 
	[identifier_type_text] nvarchar(50), 
	[extension_url] nvarchar(50), 
	[extension_valueDecimal] nvarchar(50), 
	[extension_valueString] nvarchar(50), 
	[extension_valueAddress_city] nvarchar(50), 
	[extension_valueAddress_country] nvarchar(50), 
	[extension_valueAddress_state] nvarchar(50), 
	[communication_language_text] nvarchar(50), 
	[address_city] nvarchar(50), 
	[address_country] nvarchar(50), 
	[address_postalCode] nvarchar(50), 
	[address_state] nvarchar(50), 
	[name_given] nvarchar(50), 
	[name_prefix] nvarchar(50), 
	[address_line] nvarchar(50), 
	[address_extension_url] nvarchar(50), 
	[communication_language_coding_code] nvarchar(50), 
	[communication_language_coding_display] nvarchar(50), 
	[communication_language_coding_system] nvarchar(50), 
	[identifier_type_coding_code] nvarchar(50), 
	[identifier_type_coding_display] nvarchar(50), 
	[identifier_type_coding_system] nvarchar(50), 
	[address_extension_extension_url] nvarchar(50), 
	[address_extension_extension_valueDecimal] nvarchar(50));